<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Get-ready" data-toc-modified-id="Get-ready-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Get ready</a></span><ul class="toc-item"><li><span><a href="#Install-chromedriver-mac-os-x" data-toc-modified-id="Install-chromedriver-mac-os-x-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Install chromedriver mac os x</a></span></li><li><span><a href="#permit-chromedriver" data-toc-modified-id="permit-chromedriver-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>permit chromedriver</a></span></li></ul></li><li><span><a href="#Get-newspaper-twitter-ids" data-toc-modified-id="Get-newspaper-twitter-ids-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get newspaper twitter ids</a></span><ul class="toc-item"><li><span><a href="#get-twitter-api-access-tokens,-save-and-set-up-environmental-variables" data-toc-modified-id="get-twitter-api-access-tokens,-save-and-set-up-environmental-variables-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>get twitter api access tokens, save and set up environmental variables</a></span></li><li><span><a href="#Get-relevant-twitter-handles-source:" data-toc-modified-id="Get-relevant-twitter-handles-source:-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Get relevant twitter handles source:</a></span></li><li><span><a href="#Webscrapping:-get-twitter-ids-from-the-website" data-toc-modified-id="Webscrapping:-get-twitter-ids-from-the-website-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Webscrapping: get twitter ids from the website</a></span></li></ul></li></ul></div>

In [2]:
from nltk.twitter import Query, Streamer, Twitter, TweetViewer, TweetWriter, credsfromfile
import os, sys
import time
import pickle
from time import sleep

# webscrapping
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options


## Get ready
### Install chromedriver mac os x
```
brew cask install chromedriver
```
### permit chromedriver

https://stackoverflow.com/questions/60362018/macos-catalinav-10-15-3-error-chromedriver-cannot-be-opened-because-the-de
1. Open terminal
2. Navigate to path where your chromedriver file is located
3. Execute any one of the below commands
Command1: 
```
xattr -d com.apple.quarantine <name-of-executable>
```

Example

/usr/local/Caskroom/chromedriver 

```
xattr -d com.apple.quarantine chromedriver 
```
(or)

Command2: 

```
spctl --add --label 'Approved' <name-of-executable>
```

Source: https://docwhat.org/upgrading-to-catalina

Note: This will work only with the file(s) where the above command is executed. If a new chromedriver is downloaded then the command has to be executed again on the newly downloaded file





## Get newspaper twitter ids

### get twitter api access tokens, save and set up environmental variables
```
os.environ['TWITTER']="/Users/foldername/twitter-files"
```

### Get relevant twitter handles source:

https://devexperts.com/blog/how-to-create-and-backtest-trading-strategy-on-twitter-sentiments/

In [3]:
twitter_handles = """
@business, @WSJMarkets, @WSJMoneyBeat, @stocktwits, @benzinga, @markets
, @IBDinvestors, @nytimesbusiness, @jimcramer, @TheStalwart, @ReformedBroker
, @bespokeinvest, @stlouisfed, @Wu_Tang_Finance, @StockCats, @LizAnnSonders
, @The_Real_Fly, @charliebilello, @lindayueh
, @ukarlewitz, @paulkrugman, @EIAgov, @MarketWatch, @SeekingAlpha, @zerohedge
"""
twitter_handles = [x.strip() for x in twitter_handles.split(', ')]

### Webscrapping: get twitter ids from the website
https://tweeterid.com/
    

In [14]:
#launch url
tic = time.time()

url = "https://tweeterid.com/"

options = Options()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)
driver = webdriver.Chrome()
driver.implicitly_wait(3)
driver.get(url)
for twitter_handle in twitter_handles:
    python_input = driver.find_element_by_id("twitter")
    python_button = driver.find_element_by_id("twitterButton")
    python_input.clear()
    python_input.send_keys(twitter_handle)
    python_button.click()
    sleep(5)

output_xpath='//*[@id="output"]'
soup = BeautifulSoup(driver.page_source, 'html.parser')
containers =  soup.find_all('div', {'class':"output"})
p_elements = containers[0].find_all('p')
output_tuples=[]
for p_element in p_elements[1:]:
    output_tuples.append(tuple(p_element.text.split('=> ')))
newspapers_dict = dict(output_tuples)

with open('./newspapers_dict.pkl', 'wb') as fhandle:
    pickle.dump(newspapers_dict, fhandle)


In [5]:
with open('./newspapers_dict.pkl', 'rb') as fhandle:
    newspapers_dict = pickle.load(fhandle)


In [13]:
# check the specific accounts
tw = Twitter()
# seems hard to get 100% relavant tweets
tw.tweets(follow=list(newspapers_dict.values()), limit=10, keywords='Tesla car', lang='en') # see what the newspapers are talking about

RT @business: Dubai is seeking a buyer for the cooling system operations of its biggest airport as it leans on asset sales and debt to shor…
RT @zerohedge: "I Concede Nothing": Trump Blasts "Fake News Media" For Saying He Conceded To Biden https://t.co/Ieww5GRxm1
RT @business: Peru’s interim president, Manuel Merino, says he’s resigning just six days after the impeachment of his predecessor plunged t…
President-elect Joe Biden and Vice President-elect Kamala Harris will make remarks on the U.S. economy on Monday https://t.co/7gWeR95mug
RT @zerohedge: In Striking Reversal, One Bank Warns That 2021 Could See The "Biggest Fiscal Contraction In History" https://t.co/d4PGtss8PY
RT @business: President-elect Joe Biden and Vice President-elect Kamala Harris will make remarks on the U.S. economy on Monday https://t.co…
RT @business: Dr. Fauci says the success of Pfizer’s Covid-19 vaccine trial may help persuade more people to get inoculated https://t.co/6P…
RT @zerohedge: *Trump Allies Explore